In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import gin
import model
import numpy as np
import math
import cv2
import os
import imageio
from tqdm import tqdm

2022-08-11 22:55:02.552421: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/mesa-diverted/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/mesa:/usr/lib/x86_64-linux-gnu/dri:/usr/lib/x86_64-linux-gnu/gallium-pipe
2022-08-11 22:55:02.552473: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
gin_path = './checkpoint/config_train.gin'
gin_configs = [gin_path]
with gin.unlock_config():
  gin.parse_config_files_and_bindings(
      config_files=gin_configs, bindings=None, skip_unknown=True)

seed=8
tf.random.set_seed(seed)
np.random.seed(seed)
rng = tf.random.Generator.from_seed(seed)

2022-08-11 22:55:05.788765: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/google/home/feitongtan/anaconda3/envs/py310-tf/lib/python3.10/site-packages/cv2/../../lib64:/usr/lib/mesa-diverted/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/mesa:/usr/lib/x86_64-linux-gnu/dri:/usr/lib/x86_64-linux-gnu/gallium-pipe
2022-08-11 22:55:05.788834: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-11 22:55:05.788876: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (feitongtan00.bej.corp.google.com): /proc/driver/nvidia/version does not exist
2022-08-11 22:55:05.792902: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to us

In [3]:
models, model_kwargs = model.create_models(device='cpu')
train_kwargs = model.create_train_params()

ckpt_kwargs = {
    'models': models,
}

checkpoint = tf.train.Checkpoint(**ckpt_kwargs)
checkpoint_path = './checkpoint/pretrained_model'
checkpoint.restore(checkpoint_path)

In [8]:
def parse_hdri_batch(hdri_dir):
    diffuse_hdr_map_path = os.path.join(hdri_dir, 'diffuse.hdr')
    specular_hdr_map_16_path = os.path.join(hdri_dir, 'specular_16.hdr')
    specular_hdr_map_32_path = os.path.join(hdri_dir, 'specular_32.hdr')
    specular_hdr_map_64_path = os.path.join(hdri_dir, 'specular_64.hdr')

    diffuse_hdr_map = cv2.imread(diffuse_hdr_map_path, flags=cv2.IMREAD_ANYDEPTH)
    specular_hdr_map_16 = cv2.imread(specular_hdr_map_16_path, flags=cv2.IMREAD_ANYDEPTH)
    specular_hdr_map_32 = cv2.imread(specular_hdr_map_32_path, flags=cv2.IMREAD_ANYDEPTH)
    specular_hdr_map_64 = cv2.imread(specular_hdr_map_64_path, flags=cv2.IMREAD_ANYDEPTH)
    
    hdri_batch = {
        'diffuse_hdr_map': diffuse_hdr_map,
        'specular_hdr_map_16': specular_hdr_map_16,
        'specular_hdr_map_32': specular_hdr_map_32,
        'specular_hdr_map_64': specular_hdr_map_64,
    }

    return hdri_batch

def shift_hdri(hdri_batch, pixel_shift):
    new_batch = {}
    for key in hdri_batch:
        new_batch[key] = tf.concat([hdri_batch[key][:,:,pixel_shift:],hdri_batch[key][:,:,:pixel_shift]], axis=2)
    return new_batch

In [7]:
# Rotate camera

half_num_images = 1
rendered_albedo = []
rendered_relit = []

hdri_dir = './example_hdri/birbeck_street_underpass_16k'
hdri_batch = parse_hdri_batch(hdri_dir)
for key in hdri_batch:
    hdri_batch[key] = np.expand_dims(hdri_batch[key], axis=0)

azimuths = np.linspace(-np.deg2rad(half_num_images)*5, np.deg2rad(half_num_images)*5, half_num_images * 2+1, endpoint=False)
fix_latent_code = rng.normal((1,512))
for h_shift in tqdm(azimuths, total=num_images*2+1):
    (front_rgb, front_depth, front_normal, front_masks,
    _) = models['generator'].inference_interpolate(
        fix_latent_code,
        img_size=model_kwargs['image_size'],
        fov=model_kwargs['fov'],
        ray_start=model_kwargs['ray_start'],
        ray_end=model_kwargs['ray_end'],
        num_samples=32,
        h_stddev=0.0,
        v_stddev=0.0,
        h_mean=math.pi * 0.5 + h_shift,
        v_mean=math.pi * 0.5,
        psi=0.5,
        hierarchical_num_samples=32,
        sample_dist=model_kwargs['sample_dist'],
        white_back=model_kwargs['white_back'],
        has_background=model_kwargs['has_background'],
        sample_at_infinity=model_kwargs['sample_at_infinity'],
        nerf_noise=0.0)

    (front_relit, diffuse_light,
    specular_light) = models['generator'].infer_relit(front_rgb, front_normal, front_masks,
                                        hdri_batch)

    vis_albedo = (front_rgb['high'][1][0] + 1) /2
    vis_albedo = tf.clip_by_value(vis_albedo, 0.0, 1.0)
    vis_albedo=(vis_albedo.numpy()*255).astype(np.uint8)

    vis_relit_image = (front_relit[0] + 1) /2
    vis_relit_image = tf.clip_by_value(vis_relit_image, 0.0, 1.0)
    vis_relit_image=(vis_relit_image.numpy()*255).astype(np.uint8)


    rendered_albedo.append(vis_albedo)
    rendered_relit.append(vis_relit_image)

imageio.mimsave('./albedo.gif', rendered_albedo)
imageio.mimsave('./relit.gif', rendered_relit)



    

100%|█████████████████████████████████████████████| 3/3 [00:23<00:00,  7.69s/it]


In [ ]:
# Rotate HDRI

num_images = 60
rendered_albedo = []
rendered_relit = []

hdri_dir = './example_hdri/birbeck_street_underpass_16k'
hdri_batch = parse_hdri_batch(hdri_dir)
for key in hdri_batch:
    hdri_batch[key] = np.expand_dims(hdri_batch[key], axis=0)
    
circle_percents = enumerate(np.linspace(0, 1, num_images))
fix_latent_code = rng.normal((1,512))
for _, circle_percent in tqdm(circle_percents, total=num_images):
    shift_pixel = int(circle_percent*128) % 128
    rotated_batch = shift_hdri(hdri_batch, shift_pixel)
    
    (front_rgb, front_depth, front_normal, front_masks,
    _) = models['generator'].inference_interpolate(
        fix_latent_code,
        img_size=model_kwargs['image_size'],
        fov=model_kwargs['fov'],
        ray_start=model_kwargs['ray_start'],
        ray_end=model_kwargs['ray_end'],
        num_samples=32,
        h_stddev=0.0,
        v_stddev=0.0,
        h_mean=math.pi * 0.5 + h_shift,
        v_mean=math.pi * 0.5,
        psi=0.5,
        hierarchical_num_samples=32,
        sample_dist=model_kwargs['sample_dist'],
        white_back=model_kwargs['white_back'],
        has_background=model_kwargs['has_background'],
        sample_at_infinity=model_kwargs['sample_at_infinity'],
        nerf_noise=0.0)

    (front_relit, diffuse_light,
    specular_light) = models['generator'].infer_relit(front_rgb, front_normal, front_masks,
                                        rotated_batch)

    vis_albedo = (front_rgb['high'][1][0] + 1) /2
    vis_albedo = tf.clip_by_value(vis_albedo, 0.0, 1.0)
    vis_albedo=(vis_albedo.numpy()*255).astype(np.uint8)

    vis_relit_image = (front_relit[0] + 1) /2
    vis_relit_image = tf.clip_by_value(vis_relit_image, 0.0, 1.0)
    vis_relit_image=(vis_relit_image.numpy()*255).astype(np.uint8)


    rendered_albedo.append(vis_albedo)
    rendered_relit.append(vis_relit_image)

imageio.mimsave('./albedo.gif', rendered_albedo)
imageio.mimsave('./relit.gif', rendered_relit)



    

 95%|████████████████████████████████████████▊  | 57/60 [06:38<00:21,  7.09s/it]